In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from fullfrlm import flow_refueling_location_model
from generate_network import generate_network
from flow_computation import flow_computation
from random_vessel_generator import random_vessel_generator
from first_stage_frlm import first_stage_frlm
from second_stage_frlm import second_stage_frlm
from create_input_data_abm import create_input_data_abm
from visualize_placement import visualize_placement
from get_abm_data import get_vessel_data_batch, get_cs_data_batch
import pickle
from determine_additional_nodes import determine_additional_nodes
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import time

C:\Users\Fabia\miniconda3\envs\ox\lib\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
G = pickle.load(open('data/network_cleaned_final.p', 'rb'))
df_h = pickle.load(open("data/revised_cleaning_results/harbour_data_100.p", "rb"))
df_ivs = pickle.load(open("data/revised_cleaning_results/ivs_exploded_100.p", "rb"))
path_lengths = pickle.load(open("data/revised_cleaning_results/path_lengths_ship_specific_routes.p", "rb"))
paths = pickle.load(open('data/final_paths.p', "rb"))

In [3]:
df_ivs.groupby(['origin','destination']).sum().reset_index().sort_values('trip_count', ascending=False)

,origin,destination,trip_count,hour,M12,M8,BII-6b,M10,BIIa-1,M9,...,C4,B04,C2l,BII-2L,B02,C1b,C2b,B01,C1l,route_v
120,NLRTM,NLAMS,9932,825,2076,3510,0,427,102,1316,...,5,5,7,39,4,0,0,0,0,72
136,NLRTM,NLMOE,2685,552,70,1584,0,56,2,144,...,2,0,0,1,0,0,0,0,0,24
145,NLRTM,NLVLI,2151,276,190,658,0,140,10,414,...,0,1,2,0,0,0,1,0,0,0
8,NLAMS,NLAER,2114,275,6,132,0,7,2,0,...,0,37,1,0,3,0,1,1,0,0
174,NLVEG,NLRTM,1467,276,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,NLPER,NLVLI,92,365,1,20,0,1,0,36,...,0,0,0,0,0,0,0,0,0,23
133,NLRTM,NLLAV,90,269,0,40,0,46,0,0,...,0,0,0,0,0,0,0,0,0,0
103,NLNIJ,NLOMD,87,271,0,23,0,0,30,9,...,0,0,0,20,0,0,0,0,0,0
161,NLTIE,NLVLI,81,269,0,43,0,29,0,5,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# pick seeds
import numpy as np
pick_from = np.linspace(0, 1000000, 1000001)
seeds = np.random.choice(a=pick_from, size=100, replace=False)
seeds = list(seeds)
seeds = [round(i) for i in seeds]

In [5]:
seeds

[192851,
 371193,
 156320,
 42774,
 564023,
 386760,
 780624,
 426344,
 874076,
 98930,
 277793,
 508119,
 505579,
 374666,
 11992,
 314837,
 661554,
 68156,
 405904,
 501475,
 123614,
 636170,
 284327,
 589233,
 336965,
 10364,
 145042,
 565816,
 233029,
 945701,
 349380,
 711944,
 206118,
 47019,
 452477,
 384818,
 683699,
 500264,
 555018,
 590007,
 602409,
 635303,
 337621,
 327822,
 482862,
 458723,
 275710,
 115423,
 19682,
 387205,
 374496,
 909574,
 560151,
 307041,
 825225,
 406556,
 260588,
 498575,
 54380,
 864866,
 270439,
 568011,
 288159,
 620269,
 69488,
 848783,
 423745,
 230617,
 354074,
 865052,
 619527,
 963402,
 611107,
 812032,
 47616,
 910883,
 366632,
 8970,
 747140,
 482276,
 41869,
 46033,
 22259,
 303654,
 316429,
 508825,
 658250,
 675389,
 130248,
 676693,
 234640,
 81926,
 342218,
 968753,
 563040,
 587633,
 814902,
 196538,
 446545,
 563796]

In [6]:
seeds = [259572, 7542, 864199, 11540, 188813, 411862, 546078, 420812, 304196, 814030]

In [7]:
%%time
total_flow, fraction_captured_total, serviceable_fraction, served_fraction, optimal_facilities, \
    non_zero_flows, routes_supported, paths, G, df_abm, df_random_out, max_flow = flow_refueling_location_model(1, seeds[0], 150000, 15000, 175, 100, 600, 2, 24, False, False, False, True)

KeyboardInterrupt: 

In [8]:
serviceable_fraction

NameError: name 'serviceable_fraction' is not defined

In [ ]:
non_zero_flows

In [ ]:
df_h = pickle.load(open("data/revised_cleaning_results/harbour_data_100.p", "rb"))
facilities = [i for i in optimal_facilities.keys() if optimal_facilities[i] > 0]

extra_nodes_used = list(set(facilities)-set(list(df_h.harbour_node)))

In [ ]:
print(total_flow, fraction_captured_total, serviceable_fraction, served_fraction, routes_supported)

# Complete batch run first before continuing
Run file batchrun.py in ABM/ownwork

## Rerun using feedback from ABM

In [ ]:
vessel_data_abm = get_vessel_data_batch()

In [ ]:
station_data_abm = get_cs_data_batch()

In [ ]:
station_data_abm

In [ ]:
top5_least_used = list(station_data_abm.sort_values(by='occupation').head(5).index)

In [ ]:
top5_least_used

In [ ]:
%%time
total_flow, fraction_captured_total, serveable_fraction, served_fraction, optimal_facilities, \
non_zero_flows, routes_supported, paths, G, df_abm, df_random_out = flow_refueling_location_model(load, 70000, 40, 1000, 2, df_random, False, False, True, top5_least_used)

In [ ]:
facilities = [i for i in optimal_facilities.keys() if optimal_facilities[i] > 0]
extra_nodes_used = list(set(facilities) - set(list(df_h.harbour_node)))
extra_nodes_used

In [ ]:

print(total_flow, fraction_captured_total, serveable_fraction, served_fraction, routes_supported)

## Now simulate again

## Stepwise run of all components of frlm

### Load necessary data

In [ ]:
df_h = pickle.load(open("data/revised_cleaning_results/harbour_data_100.p", "rb"))
df_ivs = pickle.load(open("data/revised_cleaning_results/ivs_exploded_100.p", "rb"))
path_lengths = pickle.load(open("data/revised_cleaning_results/path_lengths_ship_specific_routes.p", "rb"))

In [ ]:
df_ivs

## Generate random df

In [ ]:
%%time
df_chance = df_ivs.copy()
df_random1 = random_vessel_generator(df_chance, 1)

In [ ]:
df_random1


## Compute path flows

In [ ]:
%%time
flows = flow_computation(df_random1)

In [ ]:
sum(flows.values())

## Generate network

In [ ]:
%%time
G = pickle.load(open('data/network_cleaned_final.p', 'rb'))
paths = pickle.load(open("data/final_paths.p", "rb"))
G, paths, inserted_nodes = generate_network(G, paths, 60000)

In [ ]:
inserted_nodes += determine_additional_nodes(G, df_h, 60000)

In [ ]:
# now inserts 1000 nodes, goes quite wrong...

In [ ]:
inserted_nodes

In [ ]:
%%time
# range r in meters
df_b, df_g, df_eq_fq, feasible_combinations = first_stage_frlm(70000, G, OD=flows, paths=paths, path_lengths=path_lengths, df_h=df_h, additional_nodes=inserted_nodes)

## Run second stage FRLM

In [ ]:
%%time
optimal_facilities, optimal_flows, non_zero_flows, supported_flow, routes_supported = second_stage_frlm(70000, 15000, 175, 40, 2500, 2, df_g, df_b, df_eq_fq)

In [ ]:
optimal_facilities

In [ ]:
sum(optimal_facilities.values())

In [ ]:
sum(optimal_flows.values())

In [ ]:
supported_flow

## Create input data ABM

In [ ]:
%%time
df_abm = create_input_data_abm(G, paths, non_zero_flows, optimal_facilities)

In [ ]:
df_abm

## Visualize placement

In [ ]:
visualize_placement(G,flows, optimal_facilities, non_zero_flows, df_h, paths, True)